# Geração de Texto com RNN e seu Próprio Dataset

Este notebook contém o código essencial do tutorial de geração de texto do TensorFlow, adaptado para que você possa usar seu próprio arquivo de texto.

## 1. Configuração

Primeiro, importamos as bibliotecas necessárias.

In [19]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [20]:
import tensorflow as tf
import numpy as np
import time

### Carregue seu próprio texto

**Instrução:** Faça o upload do seu arquivo `.txt` para o ambiente do Colab e substitua `'seu_arquivo.txt'` pelo nome do seu arquivo.

In [27]:
path_to_file = 'carlos_drummond.txt' # <-- SUBSTITUA AQUI

# Leia o conteúdo do arquivo
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

print(f'Tamanho do texto: {len(text)} caracteres')

# Veja os primeiros 250 caracteres do seu texto
print("\n--- Início do texto ---")
print(text[:250])
print("-----------------------")

# Extrai o vocabulário (caracteres únicos)
vocab = sorted(set(text))
print(f'{len(vocab)} caracteres únicos')

Tamanho do texto: 296418 caracteres

--- Início do texto ---
Alguma Poesia
Carlos Drummond de Andrade

A Mario de Andrade, meu amigo

1. POEMA DE SETE FACES
QUANDO NASCI, um anjo torto
desses que vivem na sombra
disse: Vai, Carlos! ser gauche na vida.
As casas espiam os homens
que correm atras de mulheres.
A t
-----------------------
80 caracteres únicos


## 2. Processamento do Texto

Agora, convertemos o texto de caracteres para uma representação numérica (IDs).

In [28]:
# Cria a camada de mapeamento de caracteres para IDs
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

# Cria a camada para fazer o mapeamento inverso: de IDs para caracteres
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

# Função auxiliar para converter uma sequência de IDs de volta para texto
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

## 3. Preparação dos Dados para Treinamento

Criamos sequências de entrada e saída para que o modelo aprenda a prever o próximo caractere.

In [29]:
# Converte todo o texto em uma sequência de IDs
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))

# Cria um dataset do TensorFlow a partir dos IDs
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

# Define o tamanho da sequência para o treinamento
seq_length = 100

# Cria sequências de (seq_length + 1) caracteres
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

# Função para dividir cada sequência em entrada (input) e alvo (target)
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [30]:
# Parâmetros de treinamento
BATCH_SIZE = 64
BUFFER_SIZE = 10000

# Embaralha e agrupa os dados em lotes (batches)
dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

print(dataset)

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>


In [31]:
# Hiperparâmetros do modelo
vocab_size = len(ids_from_chars.get_vocabulary())
embedding_dim = 256
rnn_units = 1024

## 4. Construção do Modelo RNN

Definimos a arquitetura do nosso modelo usando Keras. Ele terá uma camada de Embedding, uma camada GRU (um tipo de RNN) e uma camada Densa de saída.

In [40]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__()
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)

    # IMPORTANTE: A camada GRU foi trocada por LSTM
    self.lstm = tf.keras.layers.LSTM(rnn_units,
                                   return_sequences=True,
                                   return_state=True)

    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)

    # IMPORTANTE: A chamada agora é para self.lstm
    # A LSTM retorna 3 valores: a saída e dois estados (h e c)
    x, state_h, state_c = self.lstm(x, initial_state=states, training=training)

    # Agrupamos os estados para a próxima iteração
    states = [state_h, state_c]

    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

# A instanciação do modelo agora deve funcionar sem erros
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

model.summary()

Model: "my_model_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

## 5. Treinamento do Modelo

Compilamos o modelo com uma função de perda e um otimizador, e então iniciamos o treinamento.

In [41]:
# Define a função de perda
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compila o modelo
model.compile(optimizer='adam', loss=loss)

In [42]:
# Diretório onde os checkpoints serão salvos
checkpoint_dir = './training_checkpoints'

# Nome dos arquivos de checkpoint (com a extensão correta)
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

print(f"Os checkpoints serão salvos com o prefixo: {checkpoint_prefix}")

Os checkpoints serão salvos com o prefixo: ./training_checkpoints/ckpt_{epoch}.weights.h5


In [47]:
# Define o número de épocas para o treinamento
EPOCHS = 50

# Inicia o treinamento
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])


Epoch 1/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step - loss: 1.6307
Epoch 2/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step - loss: 1.6023
Epoch 3/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 67ms/step - loss: 1.5807
Epoch 4/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 5s 67ms/step - loss: 1.5540
Epoch 5/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 5s 69ms/step - loss: 1.5415
Epoch 6/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 67ms/step - loss: 1.5206
Epoch 7/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 67ms/step - loss: 1.5012
Epoch 8/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 5s 69ms/step - loss: 1.4759
Epoch 9/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 68ms/step - loss: 1.4533
Epoch 10/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 68ms/step - loss: 1.4384
Epoch 11/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 68ms/step - loss: 1.4179
Epoch 12/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 68ms/step - loss: 1.3929
Epoch 13/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 5s 68ms/step - loss: 1.3739
Epoch 14/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 68ms/step - loss: 1.3514
Epoch 15/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 68ms/step - loss: 1.3257
Epoc

## 6. Geração de Texto

Com o modelo treinado, agora podemos usá-lo para gerar texto. Criamos um modelo de "um passo" que gera um caractere de cada vez.

In [48]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    predicted_logits = predicted_logits + self.prediction_mask

    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    predicted_chars = self.chars_from_ids(predicted_ids)
    return predicted_chars, states

one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

### Execute o loop de geração

Agora, vamos gerar 1000 caracteres. Você pode (e deve!) alterar o texto inicial (`next_char`) para ver como o modelo responde a diferentes prompts.

In [49]:
start = time.time()
states = None
next_char = tf.constant(['Mario']) # <-- MUDE O TEXTO INICIAL AQUI
result = [next_char]

for n in range(1000): # Gera 1000 caracteres
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()

print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print(f'\nTempo de execução: {end - start:.2f}s')

Mario dos serros.

o BRaVERINTA

Asti, ela forta
capri
Nao era Mariando.
Sou carguiz a sitir louco
de ris, preto, virgouio palavra.
O dia entranho (minha sentenca
da alguem certo que me cartila
em bordaces, melhor mulher
Andrade pada

O chao e cama sempre a cingil me silvo
no pais desgosto dos AmOr.
Descebo ideirava ha o outro, por
[anto.
A lingua grouxante?
Estretam-se na pureza
deste mundo enfasto,
esbou, se vai dar noite,
sua coxa se aflexo
algamo irregasoava
no sono damos o homem?
- As outras de calma porem nada.
O vidrileiro coqueir
e nao agre vosso pai.
Mas o campo e dor.
O mundo entre nos oprimidos
para decifros ficou de todos,
teu companheiro corredor;
hora que frateira
tem luz-se e o selancolica
nas cavernais de queixa
com qual Nao amadeira
minha cor de amor acordado
que a noite era nobre
vai devassando
paca-los bercos,
umas pousas.
Nao e agora, orvalhado;
so esperso e um vago
esperando
a mesa, prolquidade.
Nao contato, na conhca
no chao, no chao.
Era no mur,
em nos ou nao e n